In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
brands = pd.read_csv("/workingdir/data/brand_promotions.csv")

In [5]:
brands.head()

,brand,start_date,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29
0,7,2014-05-13,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,1
1,36,2012-04-22,0,1,0,1,1,0,1,0,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0
2,33,2015-03-03,1,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0
3,22,2013-10-09,0,1,1,0,0,1,0,1,1,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
4,11,2013-05-17,0,0,1,1,1,1,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1
